In [ ]:
import os
import numpy as np
import rasterio
import shutil
from vis_tools import read_hysplit_netcdf

FILENAME = '18042918_taupo_15.0_0.01.nc'

folder = FILENAME.rstrip('.nc')
raw_dir = os.path.join(folder, 'raw')
log_dir = os.path.join(folder, 'log10')
if not os.path.exists(raw_dir):
    os.makedirs(raw_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)    
    
# read netCDF (remove the trivial, empty t=0 step and transpose for GeoTIFF coord order)
temp = read_hysplit_netcdf(FILENAME).isel(time=slice(1, None)).transpose('time', 'lat', 'lon')

nodata = -1
np.place(temp['total_deposition'].values, np.isnan(temp['total_deposition'].values), nodata)  # transform all NaN's to nodata

temp.to_netcdf('temp.nc')
model = rasterio.open('temp.nc', 'r')
os.remove('temp.nc')

for time_step in model.indexes:
    
    timestamp = str(np.datetime64(temp['time'].values[time_step-1], 'h'))
    
    raw_fname = FILENAME.rstrip('.nc')+'_'+timestamp+'.tif'
    raw_rast = rasterio.open(os.path.join(raw_dir, raw_fname),
                                'w', driver='GTiff',
                                height=model.height, width=model.width,
                                count=1, dtype=model.profile['dtype'],
                                crs={'init': 'epsg:4326'}, transform=model.affine,
                                nodata=nodata)
    raw_rast.write(model.read(time_step), 1)  
    shared_profile = raw_rast.profile
    raw_rast.close()
    print('finished writing {}'.format(raw_fname))
    
    log_fname = FILENAME.rstrip('.nc')+'_'+timestamp+'_log10'+'.tif'
    with rasterio.open(os.path.join(log_dir, log_fname), 'w', **shared_profile) as log_rast:
        log_data = np.log10(model.read(time_step))
        np.place(log_data, np.isnan(log_data), nodata)
        log_rast.write(log_data, 1)
    print('finished writing {}\n'.format(log_fname))
    
model.close()

shutil.make_archive(os.path.join(folder, folder), 'zip', folder)

print('done')

In [ ]:
# DRAFT

import simplekml
import rasterio

FILENAME = '18042918_taupo_15.0_0.01/log10/18042918_taupo_15.0_0.01_2018-04-30T18_log10.tif'

kml = simplekml.Kml()

image = kml.newgroundoverlay(name='test')
image.icon.href = kml.addfile(FILENAME)
with rasterio.open(FILENAME, 'r') as src:
    image.latlonbox.north = src.bounds.top
    image.latlonbox.south = src.bounds.bottom
    image.latlonbox.east = src.bounds.right
    image.latlonbox.west = src.bounds.left
    
kml.savekmz('ash.kmz')